In [1]:
import fiona
import rtree
from pyproj import transformer
import geopandas
import pandas
from shapely.geometry import LineString, MultiLineString

from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColumnDataSource

In [2]:
# Read in a single strava file. Add functionality for multiple rides later
strava = geopandas.read_file('/home/jenny/Downloads/Afternoon_Ride2.gpx', layer='tracks')
strava = strava.to_crs("EPSG:3857")

In [3]:
# Turn the strava MultiLineString into short LineStrings
for x in strava['geometry'].iloc[0]:
    coords = [y for y in x.coords]
    lines = []
    for i in range(len(coords)-1):
        lines.append(LineString([coords[i], coords[i+1]]))

d = {'col1': range(len(lines)), 'geometry': lines}
strava_gdf = geopandas.GeoDataFrame(d, crs="EPSG:3857")


In [4]:
# Read in the map for the city
import osmnx as ox
#ox.geocode_to_gdf

G = ox.graph_from_place('Cambridge, MA USA', network_type='drive', simplify=True)

# Turn into a gdf in epgs 3857
nodes, ways = ox.utils_graph.graph_to_gdfs(G)
ways = ways.to_crs("EPSG:3857") 


In [5]:
# Add base map for city
output_file("tile.html")

tile_provider = get_provider(CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
p = figure(x_range=[ways.total_bounds[0], ways.total_bounds[2]], y_range=[ways.total_bounds[1], ways.total_bounds[3]])
p.add_tile(tile_provider)


TileRenderer(id='1033', ...)

In [6]:
def add_gdf_to_map(gdf, p, color, width=2):
    x_coords = []
    y_coords = []

    # display Cambridge roads
    for index, road in gdf.iterrows():
        coords = [x for x in road['geometry'].coords]
        x_coords.append([x[0] for x in coords])
        y_coords.append([x[1] for x in coords])


    # make a multilinestring from a set of linestrings
    source = ColumnDataSource( dict(x=x_coords,y=y_coords) )    
    l = p.multi_line( xs='x', ys='y', source=source, color=color, line_width=width)                    


    return p

def add_multilines_to_map(df, p, color, width=2):
    x_coords = []
    y_coords = []
    for index, element in df.iterrows():
        for line_string in element['geometry']:
            coords = [x for x in line_string.coords]
            x_coords.append([x[0] for x in coords])
            y_coords.append([x[1] for x in coords])
            
    # make a multilinestring from a set of linestrings
    source = ColumnDataSource( dict(x=x_coords,y=y_coords) )    
    l = p.multi_line( xs='x', ys='y', source=source, color=color, line_width=width)                    
    return p
            #print(coords)
# Add ways to map
p = add_gdf_to_map(ways, p, 'red')
# Add strava routes to map
p = add_multilines_to_map(strava, p, 'green', width=5)
show(p)

In [7]:
# Now make a new DataFrame with a buffer around the strava segments
#buffered = strava_gdf.buffer(20)
#buffered.head()

#coords = [x for x in buffered.iloc[21].exterior.coords]

#p.patch([x[0] for x in coords], [x[1] for x in coords], alpha=0.5, line_width=5, color='blue')
#p.patches([[x[0] for x in coords], [x[1] for x in coords]])
#buffered.head()
#for index, buffered_segment in buffered.iterrows():
    
#p.patches([[1, 3, 2], [3, 4, 6, 6]], [[2, 1, 4], [4, 7, 8, 5]],
#          color=["firebrick", "navy"], alpha=[0.8, 0.3], line_width=2)
#show(p)

In [30]:
#buffered2 = strava.buffer(20)
#foo = [x for x in buffered2.iloc[0].exterior.coords]
#foo[0:100]

#coords = [x for x in buffered2.iloc[0].exterior.coords]

#p.patch([x[0] for x in coords], [x[1] for x in coords], alpha=0.5, line_width=5, color='orange')
#show(p)

In [83]:
lines = strava.iloc[0]['geometry']

# 
buffered = lines.buffer(20)
buffered_df = geopandas.GeoDataFrame({'geometry': buffered, 'df1':[1]}, crs="EPSG:3857")


intersecting = geopandas.sjoin(ways, buffered_df, how='inner', op='intersects', lsuffix='left', rsuffix='right')
count = 0
for i, way in intersecting.iterrows():
    if buffered_df['geometry'].contains(way['geometry']).any():
        count+=1
print(count)
print(len(intersecting))

#intersecting.loc[buffered_df.contains(intersecting['geometry'])]

#gdf = geopandas.GeoDataFrame(df, geometry=df.apply(lambda row: Point(row.lon,row.lat), axis=1)

                             
                             
#buffered.intersects(ways)


#coords = [x for x in lines[0].coords]
#len(coords)

#LineString(coords[0:500]).buffer(20)

98
235


/home/jenny/anaconda3/envs/crash-model/lib/python3.6/site-packages/geopandas/base.py:39: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


,osmid,highway,oneway,length,geometry,lanes,name,maxspeed,width,ref,bridge,tunnel,access,junction,u,v,key,index_right,df1
